In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import os
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction import text 
from sklearn.decomposition import LatentDirichletAllocation as LDA
from pyLDAvis import sklearn as sklearn_lda
import pyLDAvis

# import lda


C:\Users\jjing\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\Users\jjing\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [2]:
# max rows and columns
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
# display all contents in dataframe
pd.option_context("display.max_colwidth", -1, "display.max_rows", None)

In [3]:
df = pd.DataFrame()
for file in os.listdir("../data/"):
    df = pd.concat([df, pd.read_csv("../data/"+file)], axis=0)

In [4]:
df.isnull().sum()

subreddit        0
created_utc      0
body           754
score            0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df['year'] = [datetime.fromtimestamp(utc).year for utc in df['created_utc']]
df['month'] = [datetime.fromtimestamp(utc).month for utc in df['created_utc']]

In [7]:
df['year_month'] = df['year'].map(str) + '-' + df['month'].map(str)
df['year_month'] = pd.to_datetime(df['year_month'], format='%Y-%m').dt.strftime('%Y-%m')

In [8]:
df.drop(columns=['created_utc'], axis=1, inplace=True)

In [9]:
df['subreddit'].unique()

array(['HeadphoneAdvice', 'headphones'], dtype=object)

In [10]:
df[df['subreddit']!='HeadphoneAdvice']['year'].unique()

array([2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010],
      dtype=int64)

In [11]:
df.head()

,subreddit,body,score,year,month,year_month
0,HeadphoneAdvice,[PA] Upgrade from HyperX cloud 2's.I have been...,1,2020,2,2020-02
1,HeadphoneAdvice,"Neutral, USB stick style DAC, $100-200 range. ...",1,2020,2,2020-02
2,HeadphoneAdvice,Sennheiser HD 4.5 BTNC vs JBL Live 650BTNCLook...,1,2020,2,2020-02
3,HeadphoneAdvice,Best General Use IEMS under 200?I've been look...,1,2020,2,2020-02
4,HeadphoneAdvice,What gaming headphones should I buy?I have use...,1,2020,2,2020-02


In [12]:
stopwords_final = text.ENGLISH_STOP_WORDS.union({'https', 'http', 'www', 'com', 'headphones', 'headphone', 'earphones', 'earphone'})

In [13]:
cvec = CountVectorizer(ngram_range=(1,2), min_df=5, max_df=0.9, max_features=20000, stop_words=stopwords_final)
X = cvec.fit_transform(df['body'])

In [14]:
# Create LDA visualization and return trained LDA model
def get_LDA(n_topics, training_data, vectorized):
    # Create and fit the LDA model
    model = LDA(n_components=n_topics, n_jobs=4)
    model.fit(training_data)
    
    # Create LDA visualization
    LDAvis_prepared = sklearn_lda.prepare(model, training_data, vectorized)
    pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(n_topics) +'.html')
    
    return model

In [15]:
n_topics = 7
lda_model = get_LDA(n_topics, X, cvec)

C:\Users\jjing\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [19]:
n_words = 30
top_words = {}
for i, topic in enumerate(lda_model.components_):
    words = np.array(cvec.get_feature_names())[np.argsort(topic)][:-(n_words+1):-1]
    top_words[i] = [word for word in words]
top_words

{0: ['amp',
  'dac',
  'amp x200b',
  'x200b',
  'audio',
  'fiio',
  'dac amp',
  'sound',
  'use',
  'need',
  'amazon',
  'just',
  'pro',
  'dt',
  'schiit',
  'good',
  'using',
  'power',
  'setup',
  'hd',
  'amp dac',
  'ohm',
  'dp',
  'like',
  'better',
  'usb',
  'ref',
  'magni',
  'looking',
  'sennheiser'],
 1: ['deleted',
  'fi',
  'guide',
  'akg',
  'head fi',
  'head',
  'reddit',
  'comments',
  'org',
  'amp',
  'looking',
  'fi org',
  'buying',
  'purchase',
  'reddit comments',
  'gaming',
  'fostex',
  'thread',
  'sure',
  'read',
  'check',
  'buying guide',
  'spreadsheet',
  'make',
  'format',
  'look',
  'posting',
  '50',
  'make sure',
  'google'],
 2: ['wireless',
  'ear',
  'good',
  'looking',
  'sound',
  'quality',
  'bluetooth',
  'noise',
  'earbuds',
  'sony',
  'pair',
  'like',
  'use',
  'don',
  'best',
  'bose',
  've',
  'want',
  'better',
  'sound quality',
  'really',
  'price',
  'iems',
  'just',
  'need',
  'ears',
  'phone',
  'wire

In [33]:
n_words = 30
df_top_words = pd.DataFrame()
for i, topic in enumerate(lda_model.components_):
    words = np.array(cvec.get_feature_names())[np.argsort(topic)][:-(n_words+1):-1]
    df_top_words = pd.concat([df_top_words, pd.DataFrame([word for word in words]).T], axis = 0)

In [35]:
df_top_words

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,amp,dac,amp x200b,x200b,audio,fiio,dac amp,sound,use,need,amazon,just,pro,dt,schiit,good,using,power,setup,hd,amp dac,ohm,dp,like,better,usb,ref,magni,looking,sennheiser
0,deleted,fi,guide,akg,head fi,head,reddit,comments,org,amp,looking,fi org,buying,purchase,reddit comments,gaming,fostex,thread,sure,read,check,buying guide,spreadsheet,make,format,look,posting,50,make sure,google
0,wireless,ear,good,looking,sound,quality,bluetooth,noise,earbuds,sony,pair,like,use,don,best,bose,ve,want,better,sound quality,really,price,iems,just,need,ears,phone,wired,cancelling,budget
0,just,cable,ve,pair,know,new,got,like,thanks,pads,bought,don,ear,buy,right,amazon,audio,sennheiser,time,really,good,replacement,help,use,cables,think,ll,does,way,ago
0,gaming,music,looking,headset,like,good,sound,mic,budget,audio,use,need,open,pair,want,ath,listen,sennheiser,preferred,listening,closed,don,help,used,really,ve,pc,better,amp,quality
0,volume,removed,post,gt,purchase,help,audio,usb,questions,question,output,windows,advice,thread,issue,need,use,google,problem,sound,support,left,message,right,does,device,rule,using,noise,reddit
0,sound,bass,like,just,really,good,better,ve,don,music,bit,think,sounds,treble,people,listening,great,feel,say,soundstage,hear,little,listen,quality,pretty,different,high,mids,lot,eq


In [36]:
LDAvis_prepared = sklearn_lda.prepare(lda_model, X, cvec)
pyLDAvis.save_html(LDAvis_prepared, '../lda_visualization/ldavis_prepared_'+ str(n_topics) +'.html')

C:\Users\jjing\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [46]:

for i, topic in enumerate(lda_model.exp_dirichlet_component_):
    words = np.array(cvec.get_feature_names())[np.argsort(topic)][:-(n_words+1):-1]
    top = np.sort(topic)[:-(n_words+1):-1]
    print(sum(top))

0.2270396121996299
0.26499571303854796
0.21038186511054302
0.15847252005243356
0.20447081602113773
0.17056309256505994
0.1655751695070994


In [47]:
for i, topic in enumerate(lda_model.exp_dirichlet_component_):
    print(sum(topic))

0.9962095270543332
0.991791169117312
0.9970114744392536
0.997123696872014
0.996881031570566
0.9937674451173568
0.9975916392491869


In [43]:
lda_model.exp_dirichlet_component_

array([[4.16512306e-04, 1.96792177e-04, 6.14253254e-05, ...,
        4.37597955e-05, 5.68072365e-05, 4.68505467e-05],
       [5.88816534e-05, 1.34782250e-09, 3.29859691e-05, ...,
        1.38342004e-09, 1.35751607e-09, 1.35449982e-09],
       [1.38351706e-04, 2.43898977e-10, 5.94567013e-06, ...,
        6.31680562e-06, 2.89405216e-07, 2.45344615e-10],
       ...,
       [3.36106724e-05, 2.68275072e-10, 9.48173264e-06, ...,
        5.94099380e-06, 2.73034664e-10, 3.58155057e-06],
       [3.61582273e-06, 7.85342098e-10, 1.18374337e-05, ...,
        7.93526680e-10, 7.94539985e-10, 7.94179699e-10],
       [2.10611070e-05, 1.85886970e-10, 1.35016285e-04, ...,
        8.07845487e-07, 1.64585440e-05, 4.55180568e-10]])

In [14]:
n_topics = 20
# Create and fit the LDA model
lda_model = LDA(n_components=n_topics, n_jobs=4)
lda_model.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=4,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [15]:
top_words = 20
for i, topic in enumerate(lda_model.components_):
    words = np.array(cvec.get_feature_names())[np.argsort(topic)][:-(top_words+1):-1]
    print('Topic {}: {}'.format(i, '_'.join(words)))

Topic 0: just_like_ve_got_don_know_really_time_pair_right_think_new_people_bought_try_way_did_ago_didn_ll
Topic 1: amp_amazon_dp_ref_sr_utf8_product_utf8 amp_qid_amp sr_amp qid_keywords_th_uk_amp keywords_gp_x00_psc_amp psc_gp product
Topic 2: deleted_post_review_impressions_share_flair_need_feel_message_album_rig_harman_comment_interesting_did_image_discussion_target_impressions review_spotify
Topic 3: amp_amp x200b_x200b_audio_volume_sound_use_phone_using_usb_just_music_pc_jack_work_quality_does_plug_computer_way
Topic 4: gaming_headset_mic_sound_good_pc_looking_need_use_like_games_headsets_cloud_game_quality_microphone_want_better_hyperx_music
Topic 5: iems_iem_ve_good_thanks_kz_audio_pro_price_t2_tin_like_better_heard_reviews_audeze_best_lcd_look_ll
Topic 6: pads_dt_pro_ohm_beyerdynamic_akg_990_250_dt990_dt770_earpads_dt 990_ohms_impedance_80_beyerdynamic dt_koss_770_ear pads_db
Topic 7: looking_ear_pair_good_earbuds_wireless_like_quality_need_want_don_help_thanks_budget_know_reall

In [16]:
LDAvis_prepared = sklearn_lda.prepare(lda_model, X, cvec)
pyLDAvis.save_html(LDAvis_prepared, '../lda_visualization/ldavis_prepared_'+ str(n_topics) +'.html')

C:\Users\jjing\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [18]:
n_topics = 10
# Create and fit the LDA model
lda_model_10 = LDA(n_components=n_topics, n_jobs=4)
lda_model_10.fit(X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=4,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [19]:
top_words = 20
for i, topic in enumerate(lda_model_10.components_):
    words = np.array(cvec.get_feature_names())[np.argsort(topic)][:-(top_words+1):-1]
    print('Topic {}: {}'.format(i, '_'.join(words)))

Topic 0: gaming_looking_good_headset_mic_audio_need_quality_wireless_sound_pair_help_want_like_use_music_new_don_thanks_really
Topic 1: just_people_like_ve_know_audio_ll_thanks_think_don_gt_difference_time_high_new_different_make_youtube_post_review
Topic 2: cable_pads_deleted_cables_iems_replacement_ath_iem_ear_looking_shure_kz_purchase_removed_pro_t2_advice_tin_ve_earpads
Topic 3: sony_just_pair_good_noise_bose_buy_sound_new_price_amazon_know_like_better_quality_got_cancelling_ve_bought_anc
Topic 4: ear_earbuds_bluetooth_wireless_sound_just_use_ve_phone_right_volume_work_like_noise_left_don_problem_pair_using_issue
Topic 5: removed_guide_purchase_head fi_fi_thread_head_reddit_comments_org_advice_beats_fi org_looking_buying_gaming_google_amp_purchase advice_buying guide
Topic 6: sound_bass_like_just_really_good_bit_don_better_treble_sounds_ve_feel_great_little_think_music_mids_ears_pretty
Topic 7: hd_sennheiser_sound_open_ve_better_good_dt_looking_pair_closed_price_pro_like_akg_sennhe

In [21]:
LDAvis_prepared = sklearn_lda.prepare(lda_model_10, X, cvec)
pyLDAvis.save_html(LDAvis_prepared, '../lda_visualization/ldavis_prepared_'+ str(n_topics) +'.html')

C:\Users\jjing\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [14]:
lda_X = lda.LDA(n_topics=20, random_state=42)
lda_X.fit(X)

INFO:lda:n_documents: 372008
INFO:lda:vocab_size: 20000
INFO:lda:n_words: 15007010
INFO:lda:n_topics: 20
INFO:lda:n_iter: 2000
C:\Users\jjing\Anaconda3\lib\site-packages\lda\utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -180140394
INFO:lda:<10> log likelihood: -142531918
INFO:lda:<20> log likelihood: -131203715
INFO:lda:<30> log likelihood: -128239952
INFO:lda:<40> log likelihood: -127109346
INFO:lda:<50> log likelihood: -126549389
INFO:lda:<60> log likelihood: -126215583
INFO:lda:<70> log likelihood: -125993341
INFO:lda:<80> log likelihood: -125818127
INFO:lda:<90> log likelihood: -125706881
INFO:lda:<100> log likelihood: -125617914
INFO:lda:<110> log likelihood: -125568552
INFO:lda:<120> log likelihood: -125518962
INFO:lda:<130> log likelihood: -125466264
IN

INFO:lda:<1830> log likelihood: -125095914
INFO:lda:<1840> log likelihood: -125094561
INFO:lda:<1850> log likelihood: -125102810
INFO:lda:<1860> log likelihood: -125099965
INFO:lda:<1870> log likelihood: -125093485
INFO:lda:<1880> log likelihood: -125091465
INFO:lda:<1890> log likelihood: -125093860
INFO:lda:<1900> log likelihood: -125100967
INFO:lda:<1910> log likelihood: -125095476
INFO:lda:<1920> log likelihood: -125094491
INFO:lda:<1930> log likelihood: -125097785
INFO:lda:<1940> log likelihood: -125092070
INFO:lda:<1950> log likelihood: -125093284
INFO:lda:<1960> log likelihood: -125095979
INFO:lda:<1970> log likelihood: -125095766
INFO:lda:<1980> log likelihood: -125094820
INFO:lda:<1990> log likelihood: -125100728
INFO:lda:<1999> log likelihood: -125096301


In [37]:
top_words = 20
for i, topic in enumerate(lda_X.topic_word_):
    words = np.array(cvec.get_feature_names())[np.argsort(topic)][:-(top_words+1):-1]
    print('Topic {}: {}'.format(i, '_'.join(words)))


NameError: name 'lda_X' is not defined

In [32]:
np.array(cvec.get_feature_names())

array(['00', '00 filter', '000', ..., 'zx', 'zx300', 'zxr'], dtype='<U110')

In [41]:
lda_X.doc_topic_

array([[0.00128205, 0.00128205, 0.00128205, ..., 0.15512821, 0.00128205,
        0.00128205],
       [0.00227273, 0.00227273, 0.00227273, ..., 0.00227273, 0.18409091,
        0.00227273],
       [0.00333333, 0.23666667, 0.00333333, ..., 0.17      , 0.00333333,
        0.03666667],
       ...,
       [0.025     , 0.025     , 0.025     , ..., 0.025     , 0.025     ,
        0.025     ],
       [0.00294118, 0.00294118, 0.00294118, ..., 0.00294118, 0.00294118,
        0.00294118],
       [0.00769231, 0.08461538, 0.00769231, ..., 0.00769231, 0.00769231,
        0.00769231]])

In [42]:
lda_X.components_

array([[3.47737931e-04, 2.25789190e-08, 2.14522309e-04, ...,
        2.25789190e-08, 2.25789190e-08, 2.25789190e-08],
       [1.03788164e-08, 1.03788164e-08, 1.03788164e-08, ...,
        1.03788164e-08, 1.03788164e-08, 1.03788164e-08],
       [1.12893061e-04, 1.88123748e-08, 1.88123748e-08, ...,
        1.88123748e-08, 1.88123748e-08, 1.88123748e-08],
       ...,
       [1.00094964e-04, 2.12922705e-08, 2.12922705e-08, ...,
        2.12922705e-08, 2.12922705e-08, 2.12922705e-08],
       [1.86523317e-08, 1.86523317e-08, 1.06336943e-04, ...,
        1.86523317e-08, 2.10790001e-04, 1.86523317e-08],
       [3.00628916e-08, 3.00628916e-08, 3.00628916e-08, ...,
        1.32306786e-04, 1.59363388e-04, 3.00628916e-08]])

In [25]:
tvec = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=0.9, max_features=20000, stop_words='english')
X_train_tvec = tvec.fit_transform(X_train)
X_train_df = pd.DataFrame(X_train_tvec.toarray(), columns=tvec.get_feature_names())
X_train_df.head()

,00,00 filter,000,000 hz,003,007,009,009s,01,02,02 headphone,03,04,05,06,07,08,09,0a,0aver,0aver whtgcfjdepldtbwsvfleg5px3zocnfdvxdodve,0db,0s,10,10 10,10 15,10 20,10 best,10 db,10 headphones,10 hours,10 minutes,10 output,10 pc,10 seconds,10 sound,10 times,10 year,10 years,100,100 120,100 130,100 150,100 200,100 250,100 300,100 amazon,100 amp,100 better,100 bucks,100 budget,100 cad,100 dac,100 dollar,100 dollars,100 don,100 euro,100 euros,100 good,100 headphone,100 headphones,100 just,100 like,100 looking,100 max,100 pair,100 preferably,100 price,100 range,100 really,100 removed,100 source,100 sure,100 thanks,100 usd,100 use,100 ve,100 volume,100 want,100 willing,100 worth,1000,1000 headphones,1000 xm3,1000mx2,1000mx3,1000x,1000x m3,1000xm2,1000xm3,1000xm3 bose,1000xm3 vs,1000xm3s,100hz,100i,100ish,100s,100usd,101,102,104,105,1060,108,109,10db,10ft,10k,10khz,10s,10th,10x,11,11 11,11 28,110,1100,110db,115,119,12,12 clock,12 hours,120,1200,125,1266,128,129,13,130,135,135 recommendations,1350,139,14,140,145,149,15,15 20,15 minutes,15 years,150,150 200,150 250,150 300,150 amazon,150 budget,150 cad,150 dollar,150 dollars,150 ear,150 headphones,150 looking,150 max,150 ohm,150 ohms,150 price,150 range,150 removed,150 source,150 thanks,150 usd,1500,150i,150ohm,150usd,151b0abg1uzh2sfik,151b0abg1uzh2sfik mqpkik2jd4khqpxlfydisewvoq,1540,155,159,16,16 44,16 bit,16 ohm,16 ohms,160,1600,165,169,16bit,17,170,175,1770,1770 pro,177x,179,18,180,185,19,190,192,192khz,195,1964,199,1990,1990 pro,1990 pros,1990s,1a,1am2,1db,1k,1khz,1more,1more quad,1more stylish,1more triple,1r,1rl22ot5olcfrjzmhxigv89zped1bbj2p6zxcwwry0gy,1rl22ot5olcfrjzmhxigv89zped1bbj2p6zxcwwry0gy viewform,1s,1st,1st gen,1x,20,20 30,20 40,20 amp,20 bucks,20 dollars,20 hours,20 hz,20 minutes,20 years,200,200 250,200 300,200 400,200 500,200 amp,200 aud,200 bucks,200 budget,200 cad,200 dollar,200 dollars,200 euro,200 euros,200 good,200 headphones,200 like,200 looking,200 max,200 pair,200 price,200 range,200 really,200 removed,200 source,200 thanks,200 usd,200 ve,200 willing,2000,2000s,2009,200i,200ish,200usd,201,2010,2011,2011 02,2012,2013,2013 sony,2014,2015,2016,2017,2018,2018 https,2019,202,2020,205,206,20error,20hz,20hz 20khz,20khz,20removal,20removal 20error,20s,21,210,215,215s,22,220,225,23,230,24,24 96,24 bit,24 hours,240,249,24bit,25,25 ii,250,250 300,250 600,250 ohm,250 ohms,250 source,250 usd,2500,250ohm,250ohms,250ω,256,256112744011530242,256112744011530242 285358055514701835,256112744011530242 304585848559632385,256112744011530242 363963783728857090,25s,26,260,27,270,275,28,280,280 pro,280 pros,280s,285358055514701835,285358055514701835 download,29,29 pdf,290,299,29e146,29e146 common_headphone_questions_answered,2c,2caps,2cs,2db,2f,2f 2fwww,2fwww,2i,2i2,2i4,2k,2khz,2m,2nd,2nd gen,2nd generation,2nd hand,2pin,2s,2se,2u,2v,2x,30,30 40,30 50,30 bucks,30 day,30 days,30 dollars,30 headphones,30 hours,30 http,30 minutes,30 seconds,300,300 350,300 400,300 500,300 budget,300 cad,300 dollar,300 dollars,300 headphones,300 ii,300 ohm,300 ohms,300 pair,300 price,300 range,300 removed,300 source,300 usd,3000,300ohm,300ohms,300s,300usd,304585848559632385,304585848559632385 snoo_neu,31,3100,315,32,32 80,32 bit,32 ohm,32 ohms,320,320 kbps,320kbps,320kbps mp3,325,32bit,32ohm,32ohms,33,330,336se,34,349,35,35 ii,35 word,350,350 usd,35ii,35mm,35s,36,360,363963783728857090,363963783728857090 snoo_neu,363d,37,370,373d,38,380,380 pro,39,399,3a,3a 2f,3a weekly,3d,3d audio,3d printed,3d sound,3db,3k,3m,3mm,3rd,3rd gen,3rd party,3s,3vt0mn,3vt0mn rheadphones_is_creating_an_audiophile_playlist,3x,40,40 50,40 60,40 bt,40 bucks,400,400 500,400 budget,400 headphones,400 usd,4000,400i,400s,404,40bt,40mm,40s,40x,41,42,425,429,43,439,44,44 1khz,440,449,45,450,46,47,48,48 hours,480,48khz,...,wisdom,wise,wish,witch,witcher,withstand,wm1a,woke,woman,won,won able,won break,won fall,won fit,won hear,won make,won need,won really,won sound,won turn,won use,won using,won work,wonder,wondered,wonderful,wonderfull